In [2]:
# Imports
import utils.data_exploration as de

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchmetrics.functional.classification import binary_stat_scores

In [14]:
# Load Dataframe and confusion matrix
def load_df(i):
    dataset_df = pd.read_parquet(f"parquet_data/R{i}_DB2_df.parquet")
    conf_matrix_df = pd.read_parquet(f"parquet_data/R{i}_DB2_CM_df.parquet")
    conf_matrix = np.load(f"parquet_data/R{i}_DB2_CM.npy", allow_pickle=False)
    return dataset_df, conf_matrix_df, conf_matrix
    
dataset_df, conf_matrix_df, conf_matrix = load_df(3)
conf_matrix_total = np.sum(conf_matrix, axis=0)
metrics = de.conf_matrix_metrics(conf_matrix_total)

In [15]:
print(conf_matrix_total)
metrics = de.conf_matrix_metrics(conf_matrix_total)
de.print_metrics(metrics)
de.plot_confusion_matrix(metrics)

[25683   777 34625 23421 49104]
Total windows:................84506
True Positives:...............25683
False Positives:.............. 777
True Negatives:...............34625
False Negatives:..............23421
Accuracy:.....................  71.4%
True Positive Rate:...........  52.3%
False Positive Rate:..........   2.2%
True Negative Rate:...........  97.8%
Positive Predictive Value:....  97.1%
